In [4]:
#pip install git+https://github.com/facebookresearch/segment-anything.git
#pip install git+https://github.com/prov-gigapath/prov-gigapath
#pip install git+https://github.com/facebookresearch/sam2

import timm
from PIL import Image
from torchvision import transforms
import torch, numpy as np


In [3]:
import numpy as np
np.__version__

'2.1.2'

In [1]:
import timm
timm.__version__

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.0.11'

In [6]:
import gigapath
import gigapath.slide_encoder as slide_encoder

ImportError: cannot import name 'UnencryptedCookieSessionFactoryConfig' from 'pyramid.session' (unknown location)

In [2]:
slide_encoder = gigapath.slide_encoder.create_model("hf_hub:prov-gigapath/prov-gigapath", "gigapath_slide_enc12l768d", 1536)



AttributeError: module 'gigapath' has no attribute 'slide_encoder'

In [ ]:
tile_encoder = timm.create_model("hf_hub:prov-gigapath/prov-gigapath", pretrained=True)

transform = transforms.Compose(
    [
        transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
)


In [9]:
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="/local/scratch/clmn1/sam_checkpoints/sam_vit_h_4b8939.pth")
predictor = SamPredictor(sam.cuda())

In [18]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

#size = 224
size = 10024

input_img = np.zeros((size, size, 3))
foreground = np.zeros((size, size))
foreground[50:100, 50:100] = 1

input_prompt_points = []
while len(input_prompt_points) < 5:
    x = np.random.randint(0, size)
    y = np.random.randint(0, size)
    if foreground[x, y] == 1:
        input_prompt_points.append((x, y))

input_prompt_points = np.array(input_prompt_points)
input_prompt_point_labels = np.ones(len(input_prompt_points))



predictor.set_image(input_img)
masks, _, _ = predictor.predict(input_prompt_points, input_prompt_point_labels)

print(masks.shape)


mem_allocation = torch.cuda.max_memory_allocated()
mem_allocation_mb = mem_allocation/ 1024**2
print(mem_allocation_mb)

(3, 10024, 10024)
5739.20556640625
